In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import io
import re
from tqdm.notebook import trange, tqdm

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from torch.distributions import Categorical

from torchtext.datasets import WikiText2, EnWik9, AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T
from torch.hub import load_state_dict_from_url
from torchtext.data.functional import sentencepiece_tokenizer, load_sp_model


c:\Users\ashmi\Documents\Artificial Intelligence\.venv\Lib\site-packages\torchtext\datasets\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\ashmi\Documents\Artificial Intelligence\.venv\Lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\ashmi\Documents\Artificial Intelligence\.venv\Lib\site-packages\torchte

In [2]:
ds = load_dataset("HuggingFaceTB/cosmopedia-100k", cache_dir=r"C:\Users\Downloads")

In [3]:
import pandas as pd

In [4]:
print(ds['train'][0])

# Convert the dataset to a Pandas DataFrame
df = pd.DataFrame(ds['train'])

# Save the DataFrame to a CSV file
df.to_csv(r"C:\Users\cosmopedia_100k_train.csv", index=False)

print("Dataset successfully saved to CSV.")

{'prompt': 'Here is an extract from a webpage: "What can cause my settlement offer to be delayed?\nWhen you’ve been injured in an Austin truck accident, one of the most common questions is how long it will take for the insurance company to make an offer to settle your case. The answer depends on a variety of factors.\nThe process starts with filing an insurance claim and providing evidence that shows exactly what happened during the accident and who was at fault. This can involve gathering key Austin truck accident evidence such as:\n- Medical records\n- Photographs or video footage of the crash scene\n- Witness statements\n- Other documents related to your injuries and damages.\nOnce this information has been collected by both sides, negotiations may begin between your Austin truck accident lawyer and the insurance company on how much compensation should be offered in exchange for settling the case out of court.\nIt is important to remember that every truck accident case is unique so 

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import re

# Load the data
df = pd.read_csv(r"C:\Users\cosmopedia_100k_train.csv")


df = df.drop(columns=['seed_data', 'format', 'audience', 'text_token_length', 'prompt'])


train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


def clean_text(df):
    df['text'] = df['text'].apply(lambda text: re.sub(r'\|\n|;', ' ', text.replace('"', ' ').replace('\n', ' ')).lower())
    return df


train_df = clean_text(train_df)
test_df = clean_text(test_df)

data_set_root = r"C:\Users\Downloads"
train_csv_path = os.path.join(data_set_root, "train_data.csv")
test_csv_path = os.path.join(data_set_root, "test_data.csv")

train_df.to_csv(train_csv_path, index=False, encoding="utf-8")
test_df.to_csv(test_csv_path, index=False, encoding="utf-8")

print("Data successfully split and saved to train_data.csv and test_data.csv.")


combined_csv_path = os.path.join(data_set_root, "combined_data.csv")
combined_df = pd.concat([train_df, test_df], ignore_index=True)
combined_df.to_csv(combined_csv_path, index=False, encoding="utf-8")

print("Files successfully combined into combined_data.csv.")


Data successfully split and saved to train_data.csv and test_data.csv.
Files successfully combined into combined_data.csv.


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import re
import torch
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, df):
        self.texts = df['text'].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]
dataset_train = TextDataset(train_df)
dataset_test = TextDataset(test_df)


batch_size = 64
shuffle = True


data_loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=shuffle)
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

print("DataLoader objects created successfully.")

DataLoader objects created successfully.


In [7]:
import os
import sentencepiece as spm

options = dict(

  input=r"C:\Users\ashmi\Downloads\combined_data.txt", 
  input_format="text",

  model_prefix="tok400",  
 
  model_type="bpe",
  vocab_size=384613, 

  normalization_rule_name="identity", 
  remove_extra_whitespaces=False,
  input_sentence_size=100059,  
  max_sentence_length=4192,  
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,

  character_coverage=0.99995,
  byte_fallback=True,
  
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
 
  unk_id=0,  
  bos_id=1,  
  eos_id=2,
  pad_id=-1,

  num_threads=os.cpu_count(), 
)


spm.SentencePieceTrainer.train(**options)


In [8]:
import sentencepiece as spm


model_path = "tok400.model" 
sp = spm.SentencePieceProcessor(model_file=r"C:\Users\ashmi\tok400.model")

def tokenize_and_convert(sentence):
    
    tokens = sp.encode(sentence, out_type=str)  
    

    token_ids = sp.encode(sentence, out_type=int) 
    
    return tokens, token_ids


sentence = "hi how are you"

tokens, token_ids = tokenize_and_convert(sentence)


print("Original Sentence:", sentence)
print("Tokenized Sentence:", tokens)
print("Token IDs:", token_ids)


Original Sentence: hi how are you
Tokenized Sentence: ['▁hi', '▁how', '▁are', '▁you']
Token IDs: [5305, 472, 527, 352]


In [10]:
vocab_size

795022

In [11]:
sentence_count 


100059